In [1]:
import torch
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
_ = tokenizer.add_special_tokens(special_tokens)

questions = ["hi how are you"]
answers = ["i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")

In [2]:
questions = ["hi how are you"]
answers = ["i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")

print(inputs)
print(outputs)

{'input_ids': tensor([[ 5303,   703,   389,   345, 50257, 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0]])}
{'input_ids': tensor([[   72,   716,  1016,  3734, 50257]]), 'attention_mask': tensor([[1, 1, 1, 1, 0]])}


In [1]:
import torch
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
_ = tokenizer.add_special_tokens(special_tokens)

questions = ["hi how are you"]
answers = ["i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")

questions = ["hi how are you"]
answers = ["<|sep|> i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")

print(inputs)
print(outputs)

{'input_ids': tensor([[ 5303,   703,   389,   345, 50257, 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0]])}
{'input_ids': tensor([[50258,    72,   716,  1016,  3734]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


In [3]:
import torch
from transformers import GPT2Config
from transformers import GPT2Model, GPT2LMHeadModel

## word_embedding
# default GPT2 word embedding
gpt2configuration = GPT2Config()
word_embedder = GPT2Model(gpt2configuration)   
word_embedder.resize_token_embeddings(len(tokenizer))
word_embedder = word_embedder.wte  

In [4]:
question_embeds = word_embedder(inputs['input_ids'])
question_attention_mask = inputs['attention_mask']

answer_embeds = word_embedder(outputs['input_ids'])
answer_attention_mask = outputs['attention_mask']

print(question_embeds.shape)
print(question_attention_mask)
print(answer_embeds.shape)
print(answer_attention_mask)

torch.Size([1, 8, 768])
tensor([[1, 1, 1, 1, 0, 0, 0, 0]])
torch.Size([1, 5, 768])
tensor([[1, 1, 1, 1, 0]])


In [15]:
question_id_type = torch.zeros(*question_embeds.size()[:-1], dtype=torch.long)
question_position_id = torch.arange(0,question_embeds.size()[1])
question_position_id = torch.unsqueeze(question_position_id,0)
question_position_id = question_position_id.repeat(question_embeds.size()[0], 1)
# question_position_id = question_position_id.to(device)

answer_id_type = torch.zeros(*answer_embeds.size()[:-1], dtype=torch.long)
answer_position_id = torch.arange(0,answer_embeds.size()[1])
answer_position_id = torch.unsqueeze(answer_position_id,0)
answer_position_id = answer_position_id.repeat(answer_embeds.size()[0], 1)
# answer_position_id = answer_position_id.to(device)

print(question_id_type)
print(question_position_id, len(question_position_id[0]))
print(answer_id_type)
print(answer_position_id)

tensor([[0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[0, 1, 2, 3, 4, 5, 6, 7]]) 8
tensor([[0, 0, 0, 0, 0]])
tensor([[0, 1, 2, 3, 4]])


In [14]:
answer_position_id += 8
print(answer_position_id)

tensor([[ 8,  9, 10, 11, 12]])


In [ ]:
visual_embeds = self.visual_embedder(img_feature)
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float)
        visual_attention_mask = visual_attention_mask.to(device)

        if self.vis_pos_emb == 'zeroes':
            visual_id_type = torch.ones(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
            visual_position_id = torch.zeros(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
        elif self.vis_pos_emb == 'pos':
            visual_id_type = torch.ones(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
            visual_position_id = torch.arange(0,visual_embeds.size()[1])
            visual_position_id = torch.unsqueeze(visual_position_id,0)
            visual_position_id = visual_position_id.repeat(visual_embeds.size()[0], 1)
            visual_position_id = visual_position_id.to(device)

        
        ## question embedding:
        input['input_ids'] = input['input_ids'].to(device)
        input['attention_mask'] = input['attention_mask'].to(device)

        
        output['input_ids'] = output['input_ids'].to(device)
        output['attention_mask'] = output['attention_mask'].to(device)

        question_embeds = self.word_embedder(input['input_ids'])
        question_attention_mask = input['attention_mask']

        answer_embeds = self.word_embedder(output['input_ids'])
        answer_attention_mask = output['attention_mask']
        
        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            question_id_type = torch.zeros(*question_embeds.size()[:-1], dtype=torch.long, device=device)
            question_position_id = torch.arange(0,question_embeds.size()[1])
            question_position_id = torch.unsqueeze(question_position_id,0)
            question_position_id = question_position_id.repeat(question_embeds.size()[0], 1)
            question_position_id = question_position_id.to(device)

            answer_id_type = torch.zeros(*answer_embeds.size()[:-1], dtype=torch.long, device=device)
            answer_position_id = torch.arange(0,answer_embeds.size()[1])
            answer_position_id = torch.unsqueeze(answer_position_id,0)
            answer_position_id = answer_position_id.repeat(answer_embeds.size()[0], 1)
            answer_position_id = answer_position_id.to(device)

            
        
        
        ## combine visual and question embeds
        ## vision first
        # inputs_embeds = torch.cat((visual_embeds, question_embeds), dim=1)
        # attention_mask = torch.cat((visual_attention_mask, question_attention_mask), dim=1)

        # if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
        #     token_type_ids = torch.cat((visual_id_type, question_id_type), dim=1)
        #     position_ids = torch.cat((visual_position_id, question_position_id), dim=1)

        ## question first
        inputs_embeds = torch.cat((question_embeds, visual_embeds), dim=1)
        attention_mask = torch.cat((question_attention_mask, visual_attention_mask), dim=1)

        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            token_type_ids = torch.cat((question_id_type, visual_id_type), dim=1)
            position_ids = torch.cat((question_position_id, visual_position_id), dim=1)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

from transformers import  VisualBertConfig, GPT2Config
from transformers import VisualBertModel, GPT2Model, ViTModel, SwinModel
from transformers import GPT2LMHeadModel

device = torch.device("cpu")


''' Early Fusion GPT with CNN/Transformers'''

class EFVLEGPT2RS18Sentence(nn.Module):
    def __init__(self, model_subver = 'v3', tokenizer_len=50257, vis_pos_emb = None):
        super(EFVLEGPT2RS18Sentence, self).__init__()
        '''
        v0: visual embedding : Default patch1 + embedding form VB + GPT2 decoder
        v1: visual embedding : Default patch1 + from nn.linear    + GPT2 decoder
        v2: visual embedding : visual patches + embedding form VB + GPT2 decoder
        v3: visual embedding : visual patches + from nn.linear    + GPT2 decoder
        '''
        
        self.sub_ver = model_subver
        self.vis_pos_emb = vis_pos_emb
        
        ## image processing
        self.img_feature_extractor = models.resnet18(pretrained=True)
        if self.sub_ver == 'v0' or self.sub_ver =='v1':
            new_fc = nn.Sequential(*list(self.img_feature_extractor.fc.children())[:-1])
            self.img_feature_extractor.fc = new_fc
        elif self.sub_ver == 'v2' or self.sub_ver =='v3':
            self.img_feature_extractor = torch.nn.Sequential(*(list(self.img_feature_extractor.children())[:-2]))
        
        ## Visual_embedding
        if self.sub_ver == 'v0' or self.sub_ver =='v2':
            # visual bert embedding
            VB_config = VisualBertConfig.from_pretrained("uclanlp/visualbert-vqa-coco-pre")
            VB_config.visual_embedding_dim = 512
            visualbert = VisualBertModel(config=VB_config)
            self.visual_embedder = visualbert.embeddings.visual_projection
        elif self.sub_ver == 'v1' or self.sub_ver =='v3':
            self.visual_embedder = nn.Linear(512, 768)

        ## word_embedding
        # default GPT2 word embedding
        gpt2configuration = GPT2Config()
        word_embedder = GPT2Model(gpt2configuration)
        word_embedder.resize_token_embeddings(tokenizer_len)
        self.word_embedder = word_embedder.wte

        ## GPT2 visual context aware decoder
        self.VCAdecoder = GPT2LMHeadModel.from_pretrained('gpt2')
        
        
    def forward(self, question, img, answer):
        
        ## image encoder features
        img_feature = self.img_feature_extractor(img)
        
        if self.sub_ver == 'v0' or self.sub_ver =='v1':
            img_feature = torch.unsqueeze(img_feature, dim=1)
        if self.sub_ver == 'v2'or self.sub_ver =='v3':
            img_feature = torch.flatten(img_feature, start_dim=2)
            print(img_feature.shape)
            img_feature = img_feature.permute((0,2,1))
        
        
        ## visual Embedding : id type 1, pos: zero / incremental
        visual_embeds = self.visual_embedder(img_feature)
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float)
        visual_attention_mask = visual_attention_mask.to(device)

        if self.vis_pos_emb == 'zeroes':
            visual_id_type = torch.ones(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
            visual_position_id = torch.zeros(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
        elif self.vis_pos_emb == 'pos':
            visual_id_type = torch.ones(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
            visual_position_id = torch.arange(0,visual_embeds.size()[1])
            visual_position_id = torch.unsqueeze(visual_position_id,0)
            visual_position_id = visual_position_id.repeat(visual_embeds.size()[0], 1)
            visual_position_id = visual_position_id.to(device)
        visual_len = len(visual_position_id[0])

        
        ## question embedding:
        question['input_ids'] = question['input_ids'].to(device)
        question_embeds = self.word_embedder(question['input_ids'])
        question_attention_mask = question['attention_mask'].to(device)

        ## answer embedding        
        answer['input_ids'] = answer['input_ids'].to(device)        
        answer_embeds = self.word_embedder(answer['input_ids'])
        answer_attention_mask = answer['attention_mask'].to(device)
        
        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            question_id_type = torch.zeros(*question_embeds.size()[:-1], dtype=torch.long, device=device)
            question_position_id = torch.arange(0,question_embeds.size()[1])
            question_position_id = torch.unsqueeze(question_position_id,0)
            question_position_id = question_position_id.repeat(question_embeds.size()[0], 1)
            question_position_id = question_position_id.to(device)
            question_len = len(question_position_id[0])

            answer_id_type = torch.zeros(*answer_embeds.size()[:-1], dtype=torch.long, device=device)
            answer_position_id = torch.arange(0,answer_embeds.size()[1])
            answer_position_id = torch.unsqueeze(answer_position_id,0)
            answer_position_id = answer_position_id.repeat(answer_embeds.size()[0], 1)
            answer_position_id += (question_len+visual_len)
            answer_position_id = answer_position_id.to(device)
            
        print(question_len, visual_len, len(answer_position_id[0]))

        ## combine visual and question embeds
        ## vision first
        # inputs_embeds = torch.cat((visual_embeds, question_embeds), dim=1)
        # attention_mask = torch.cat((visual_attention_mask, question_attention_mask), dim=1)

        # if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
        #     token_type_ids = torch.cat((visual_id_type, question_id_type), dim=1)
        #     position_ids = torch.cat((visual_position_id, question_position_id), dim=1)

        ## question first
        inputs_embeds = torch.cat((question_embeds, visual_embeds, answer_embeds), dim=1)
        attention_mask = torch.cat((question_attention_mask, visual_attention_mask, answer_attention_mask), dim=1)

        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            token_type_ids = torch.cat((question_id_type, visual_id_type, answer_id_type), dim=1)
            position_ids = torch.cat((question_position_id, visual_position_id, answer_position_id), dim=1)


        ## VCA_GPT2 decoder
        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            out = self.VCAdecoder(inputs_embeds=inputs_embeds, attention_mask=attention_mask, position_ids = position_ids, token_type_ids = token_type_ids)
        else:
            out = self.VCAdecoder(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
        

        return out

In [4]:
import torch
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
_ = tokenizer.add_special_tokens(special_tokens)

questions = ["hi how are you"]
answers = ["i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")
image = torch.rand(1,3,300,256)

In [22]:
token_len = len(tokenizer)
model = EFVLEGPT2RS18Sentence(model_subver = 'v3', tokenizer_len= token_len, vis_pos_emb ='zeroes')

In [23]:
out = model(inputs, image, outputs)

torch.Size([1, 512, 80])
8 80 5


In [17]:
print(out[0].shape)

torch.Size([1, 93, 50257])


In [1]:
import torch

In [2]:
a = torch.rand(2,8)
print(a)

shift_logits = a[..., 3:-1]
shift_labels = a[..., 3+1:]
        
print(shift_logits)
print(shift_labels)


tensor([[0.9027, 0.6617, 0.0132, 0.4579, 0.3671, 0.5998, 0.6740, 0.7606],
        [0.5817, 0.6773, 0.6781, 0.1662, 0.7119, 0.9704, 0.9840, 0.0328]])
tensor([[0.4579, 0.3671, 0.5998, 0.6740],
        [0.1662, 0.7119, 0.9704, 0.9840]])
tensor([[0.3671, 0.5998, 0.6740, 0.7606],
        [0.7119, 0.9704, 0.9840, 0.0328]])


In [1]:
import torch
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
_ = tokenizer.add_special_tokens(special_tokens)

questions = ["hi how are you"]
answers = ["i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")

questions = ["hi how are you"]
answers = ["<|sep|> i am going fine"]

inputs = tokenizer(questions, padding="max_length",max_length= 8, return_tensors="pt")
outputs = tokenizer(answers, padding="max_length",max_length= 5, return_tensors="pt")

print(inputs)
print(outputs)

{'input_ids': tensor([[ 5303,   703,   389,   345, 50257, 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0]])}
{'input_ids': tensor([[50258,    72,   716,  1016,  3734]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


In [2]:
shift_labels = outputs['input_ids'][..., 1:].contiguous()
print(shift_labels)

tensor([[  72,  716, 1016, 3734]])


In [6]:
a = torch.rand(2,4,4)
print(a)
_, p_id = torch.max(a, dim=2)
print(p_id)

tensor([[[0.9419, 0.6827, 0.3752, 0.0465],
         [0.3533, 0.7009, 0.3063, 0.1070],
         [0.1069, 0.2175, 0.2988, 0.5261],
         [0.2222, 0.0170, 0.1004, 0.9328]],

        [[0.2506, 0.2457, 0.4110, 0.8580],
         [0.8879, 0.3518, 0.6792, 0.4434],
         [0.8882, 0.4520, 0.4354, 0.1541],
         [0.5476, 0.1973, 0.4182, 0.2564]]])
tensor([[0, 1, 3, 3],
        [3, 0, 0, 0]])


In [10]:
answers_GT = ["<|sep|> hi how are you", "<|sep|> i am doing fine"]
answers_Gen = ["hi how are you", "i am doing fine"]

labels = tokenizer(answers_GT, padding="max_length",max_length= 8, return_tensors="pt")
logits = tokenizer(answers_Gen, padding="max_length",max_length= 5, return_tensors="pt")


shift_labels = labels['input_ids'][..., 1:].contiguous() # 1 because answer has '<|sep|>' in front
logits = labels['input_ids'].contiguous() # 1 because answer has '<|sep|>' in front


answer_GT_dec = tokenizer.batch_decode(shift_labels, skip_special_tokens= True)
print(answer_GT_dec)
answer_Gen_dec = tokenizer.batch_decode(logits, skip_special_tokens= True)
print(answer_GT_dec)

references = []
hypotheses = []

for answer_GT_dec_i in answer_GT_dec: references.append([answer_GT_dec_i.split()])
print(references)

# Hypotheses
for answer_Gen_dec_i in answer_Gen_dec: hypotheses.append(answer_Gen_dec_i.split())
print(hypotheses)

from nltk.translate.bleu_score import corpus_bleu

# Calculate BLEU1~4
metrics = {}
metrics["Bleu_1"] = corpus_bleu(references, hypotheses, weights=(1.00, 0.00, 0.00, 0.00))
metrics["Bleu_2"] = corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0.00, 0.00))
metrics["Bleu_3"] = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.00))
metrics["Bleu_4"] = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))

print("BLEU-1 {:.6f} BLEU2 {:.6f} BLEU3 {:.6f} BLEU-4 {:.6f}".format
          (metrics["Bleu_1"],  metrics["Bleu_2"],  metrics["Bleu_3"],  metrics["Bleu_4"]))

['hi how are you', 'i am doing fine']
['hi how are you', 'i am doing fine']


[[['hi', 'how', 'are', 'you']], [['i', 'am', 'doing', 'fine']]]
[['hi', 'how', 'are', 'you'], ['i', 'am', 'doing', 'fine']]


In [14]:
from nltk.translate.bleu_score import corpus_bleu

# Calculate BLEU1~4
metrics = {}
metrics["Bleu_1"] = corpus_bleu(references, hypotheses, weights=(1.00, 0.00, 0.00, 0.00))
metrics["Bleu_2"] = corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0.00, 0.00))
metrics["Bleu_3"] = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.00))
metrics["Bleu_4"] = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))

print("BLEU-1 {:.6f} BLEU2 {:.6f} BLEU3 {:.6f} BLEU-4 {:.6f}".format
          (metrics["Bleu_1"],  metrics["Bleu_2"],  metrics["Bleu_3"],  metrics["Bleu_4"]))

ModuleNotFoundError: No module named 'nltk'

In [7]:
import glob
#train_seq  = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
seq    = [1, 5, 16]
folder_head = '../dataset/EndoVis-18-VQA/seq_'
folder_tail = '/vqa2/Sentence/*.txt'

filenames = []
for curr_seq in seq: filenames = filenames + glob.glob(folder_head + str(curr_seq) + folder_tail)
vqas = []#self.vqas = []
for file in filenames:
    file_data = open(file, "r")
    lines = [line.strip("\n") for line in file_data if line != "\n"]
    file_data.close()
    for line in lines: 
        q_type, q_s, an_s = line.split('|')
        q_s = q_s.split('&')
        an_s = an_s.split(('&'))
        for i in range(len(q_s)):
            q_a = q_s[i]+'|'+an_s[i]
            print(file, q_a)
            vqas.append([file, q_a]) #self.vqas.append([file, q_a])
print('Total files: %d | Total question: %.d' %(len(filenames), len(vqas)))

../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame110_QA.txt Which organ is being operated?|The organ being operated is kidney.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame110_QA.txt What is the organ of interest?|kidney is the organ of interest.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame110_QA.txt What is the name of the organ?|The organ name is kidney.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame110_QA.txt What tools are present in the scene?|The tools present in the scene are bipolar forceps, prograsp forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame110_QA.txt What instruments are currently used for operating the organ?|The instruments used for operating the organs are bipolar forceps, prograsp forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame110_QA.txt What is the state of bipolar forceps?|The state of bipolar forceps is Idle.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame

../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame011_QA.txt What tools are present in the scene?|The tools present in the scene are bipolar forceps, prograsp forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame011_QA.txt What instruments are currently used for operating the organ?|The instruments used for operating the organs are bipolar forceps, prograsp forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame011_QA.txt What is the state of bipolar forceps?|The state of bipolar forceps is Idle.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame011_QA.txt What is bipolar forceps doing?|The bipolar forceps is Idle.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame011_QA.txt What is the action of bipolar forceps?|The action of the bipolar forceps is Idle.
../dataset/EndoVis-18-VQA/seq_1/vqa2/Sentence/frame011_QA.txt What is the state of prograsp forceps?|The state of prograsp forceps is Tissue Manipulation.
../d

../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame135_QA.txt Which organ is being operated?|The organ being operated is kidney.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame135_QA.txt What is the organ of interest?|kidney is the organ of interest.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame135_QA.txt What is the name of the organ?|The organ name is kidney.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame135_QA.txt What tools are present in the scene?|The tools present in the scene are bipolar forceps, prograsp forceps, monopolar curved scissors and suction.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame135_QA.txt What instruments are currently used for operating the organ?|The instruments used for operating the organs are bipolar forceps, prograsp forceps, monopolar curved scissors and suction.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame135_QA.txt What is the state of bipolar forceps?|The state of bipolar forceps is Tissue Manipulation.
../dataset/EndoVis

../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame032_QA.txt What is the location of bipolar forceps?|The location of bipolar forceps is left-bottom.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame032_QA.txt Where is bipolar forceps?|The bipolar forcepsis at left-bottom.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame032_QA.txt Where is monopolar curved scissors located?|The monopolar curved scissors is located at right-bottom.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame032_QA.txt What is the location of monopolar curved scissors?|The location of monopolar curved scissors is right-bottom.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame032_QA.txt Where is monopolar curved scissors?|The monopolar curved scissorsis at right-bottom.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame052_QA.txt Which organ is being operated?|The organ being operated is kidney.
../dataset/EndoVis-18-VQA/seq_5/vqa2/Sentence/frame052_QA.txt What is the organ of interest?|kidney is the orga

../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt Which organ is being operated?|The organ being operated is kidney.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt What is the organ of interest?|kidney is the organ of interest.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt What is the name of the organ?|The organ name is kidney.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt What tools are present in the scene?|The tools present in the scene are bipolar forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt What instruments are currently used for operating the organ?|The instruments used for operating the organs are bipolar forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt What is the state of bipolar forceps?|The state of bipolar forceps is Idle.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame085_QA.txt What is bipolar fo

../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt Which organ is being operated?|The organ being operated is kidney.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt What is the organ of interest?|kidney is the organ of interest.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt What is the name of the organ?|The organ name is kidney.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt What tools are present in the scene?|The tools present in the scene are bipolar forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt What instruments are currently used for operating the organ?|The instruments used for operating the organs are bipolar forceps and monopolar curved scissors.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt What is the state of bipolar forceps?|The state of bipolar forceps is Idle.
../dataset/EndoVis-18-VQA/seq_16/vqa2/Sentence/frame120_QA.txt What is bipolar fo

In [14]:
vqas[0][1]
question, answer = vqas[0][1].split('|')
print(question)
print(answer)

Which organ is being operated?
The organ being operated is kidney.


In [15]:
answer = '<|sep|> '+answer

In [16]:
answer

'<|sep|> The organ being operated is kidney.'